In [2]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import datetime as dt
from config import postgres_password

In [ ]:
csv_file = "../Data/us_perm_visas.csv"
visa_data_df = pd.read_csv(csv_file)
visa_data_df.head()

In [ ]:
# Keep Important Columns
small_col_visa_df = visa_data_df[["case_no",
                                  "case_number",
                                  "case_status",
                                  "class_of_admission",
                                  "country_of_citizenship",
                                  "country_of_citzenship",
                                  "decision_date",
                                  "employer_country",
                                  "employer_num_employees",
                                  "employer_postal_code",
                                  "employer_state",
                                  "employer_yr_estab",
                                  "foreign_worker_info_birth_country",
                                  "fw_info_birth_country",
                                  "foreign_worker_info_education",
                                  "foreign_worker_info_major",
                                  "foreign_worker_info_postal_code",
                                  "fw_info_postal_code",
                                  "foreign_worker_info_rel_occup_exp",
                                  "fw_info_rel_occup_exp",
                                  "foreign_worker_info_state",
                                  "foreign_worker_info_training_comp",
                                  "fw_info_training_comp",
                                  "foreign_worker_ownership_interest",
                                  "fw_ownership_interest",
                                  "foreign_worker_yr_rel_edu_completed",
                                  "fw_info_yr_rel_edu_completed",
                                  "ji_offered_to_sec_j_foreign_worker",
                                  "ji_offered_to_sec_j_fw",
                                  "job_info_education",
                                  "job_info_work_postal_code",
                                  "job_info_work_state",
                                  "orig_file_date",
                                  "pw_amount_9089",
                                  "pw_unit_of_pay_9089",
                                  "wage_offer_from_9089",
                                  "wage_offer_unit_of_pay_9089",
                                  "wage_offered_from_9089",
                                  "wage_offered_unit_of_pay_9089"]]
small_col_visa_df.head()

# Combine Similar Columns

In [ ]:
#Case Number
small_col_visa_df['case_number'].fillna(small_col_visa_df['case_no'], inplace = True)
small_col_visa_df['case_number'].head()


In [ ]:
#Country of Citizenship
small_col_visa_df['country_of_citizenship'].fillna(small_col_visa_df['country_of_citzenship'], inplace = True)


In [ ]:
#FW Birth Country
small_col_visa_df['foreign_worker_info_birth_country'].fillna(small_col_visa_df['fw_info_birth_country'], inplace = True)


In [ ]:
#FW Postal Code
small_col_visa_df['foreign_worker_info_postal_code'].fillna(small_col_visa_df['fw_info_postal_code'], inplace = True)


In [ ]:
#FW Has Required Experience?
small_col_visa_df['foreign_worker_info_rel_occup_exp'].fillna(small_col_visa_df['fw_info_rel_occup_exp'], inplace = True)


In [ ]:
#FW completed Required Training?
small_col_visa_df['foreign_worker_info_training_comp'].fillna(small_col_visa_df['fw_info_training_comp'], inplace = True)


In [ ]:
#FW has ownership interest?
small_col_visa_df['foreign_worker_ownership_interest'].fillna(small_col_visa_df['fw_ownership_interest'], inplace = True)


In [ ]:
#FW has required Education?
small_col_visa_df['foreign_worker_yr_rel_edu_completed'].fillna(small_col_visa_df['fw_info_yr_rel_edu_completed'], inplace = True)


In [ ]:
#FW is position being sponsored?
small_col_visa_df['ji_offered_to_sec_j_foreign_worker'].fillna(small_col_visa_df['ji_offered_to_sec_j_fw'], inplace = True)


In [ ]:
# Wage offered
small_col_visa_df['wage_offer_from_9089'].fillna(small_col_visa_df['wage_offered_from_9089'], inplace = True)


In [ ]:
# Unit of Pay for wage offered
small_col_visa_df['wage_offer_unit_of_pay_9089'].fillna(small_col_visa_df['wage_offered_unit_of_pay_9089'], inplace = True)


# Attempting to calculate an adjusted Yearly wage

In [ ]:
#How are they paid?
small_col_visa_df.wage_offer_unit_of_pay_9089.unique()

In [ ]:
for index, row in small_col_visa_df.iterrows():
    if row["wage_offer_unit_of_pay_9089"] == "hr":
        row["Adj_annual_wage"] = row["wage_offer_from_9089"] * 40 * 50
        
        # 40 hrs/ week , 50weeks/year
        
        #Goal would be to contiue this for the other wages
        

In [ ]:
small_col_visa_df[["Adj_annual_wage"]].head()

In [ ]:
# Keep Columns -> We can remove the other related columns because they are now duplicates
# case_number
# country_of_citizenship
# foreign_worker_info_birth_country
# foreign_worker_info_postal_code
# foreign_worker_info_rel_occup_exp
# foreign_worker_info_training_comp
# foreign_worker_ownership_interest
# foreign_worker_yr_rel_edu_completed
# ji_offered_to_sec_j_foreign_worker
# wage_offer_from_9089
# wage_offer_unit_of_pay_9089




In [ ]:
slim_visa_db =  small_col_visa_df[["case_number",
                                  "case_status",
                                  "class_of_admission",
                                  "country_of_citizenship",
                                  
                                  "decision_date",
                                  "employer_country",
                                  "employer_num_employees",
                                  "employer_postal_code",
                                  "employer_state",
                                  "employer_yr_estab",
                                  "foreign_worker_info_birth_country",
                                  
                                  "foreign_worker_info_education",
                                  "foreign_worker_info_major",
                                  "foreign_worker_info_postal_code",
                                  
                                  "foreign_worker_info_rel_occup_exp",
                                  
                                  "foreign_worker_info_state",
                                  "foreign_worker_info_training_comp",
                                  
                                  "foreign_worker_ownership_interest",
                                  
                                  "foreign_worker_yr_rel_edu_completed",
                                  
                                  "ji_offered_to_sec_j_foreign_worker",
                                  
                                  "job_info_education",
                                  "job_info_work_postal_code",
                                  "job_info_work_state",
                                  "orig_file_date",
                                  "pw_amount_9089",
                                  "pw_unit_of_pay_9089",
                                  "wage_offer_from_9089",
                                  "wage_offer_unit_of_pay_9089"]]
slim_visa_db.head()

In [ ]:
slim_visa_db.set_index('case_number')

In [ ]:
rds_connection_string = f"postgres:{postgres_password}@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
engine.execute('DELETE from "visa_data"')
result = engine.execute('SELECT * FROM "visa_data"')

slim_visa_db.to_sql(name='visa_data', con=engine, if_exists='append', index=False)




In [ ]:
# Attempts to clean the data

In [ ]:
#Case Number

for index, row in small_col_visa_df.iterrows():
    df['case_number'].fillna(df['case_no'])
    
    if pd.isnull(row['case_number']):
        row['caseNumber'] = row['case_no']

    elif pd.notnull(row['case_number']):
        row['caseNumber'] = row['case_number']
        
    elif pd.isnull(row['country_of_citizenship']):
        row['countryCitizenship'] = row['country_of_citzenship']

    else:
        row['countryCitizenship'] = row['country_of_citizenship']
    
        
small_col_visa_df[["country_of_citizenship","country_of_citzenship","countryCitizenship"]].head()
     
# pd.notnull(row['country_of_citizenship'])

In [ ]:
#Country of Citizenship

for index, row in small_col_visa_df.iterrows():
    if pd.isnull(row['country_of_citizenship']):
        row['countryCitizenship'] = row['country_of_citzenship']

    else:
        row['countryCitizenship'] = row['country_of_citizenship']
        
small_col_visa_df[["country_of_citizenship","country_of_citzenship","countryCitizenship"]].head()
     

In [ ]:
#Foreign Worker Birth Country

for index, row in small_col_visa_df.iterrows():
    if pd.isnull(row['foreign_worker_info_birth_country']):
        row['birthCountry'] = row['fw_info_birth_country']

    else:
        row['birthCountry'] = row['foreign_worker_info_birth_country']
        
small_col_visa_df[["foreign_worker_info_birth_country", "fw_info_birth_country", "birthCountry"]].head()
     